In [13]:
import re, sys, math, json, os, urllib.request
import numpy as np                                
import pandas as pd                               
import matplotlib.pyplot as plt                   
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime                 

#Loading data from DataTurks
try:
  model_data = pd.read_json('data.json',
                        lines=True,
                        orient='columns')
except Exception as e:
    print('Data load error: ',e)

In [14]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
import keras
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [15]:
raw_data = model_data
raw_data.head()

,content,annotation,extras,metadata
0,"an,title","{'labels': ['N/A'], 'note': ''}",NaN,"{'first_done_at': 1529677325000, 'last_updated..."
1,"STARKY0020180125ee1o0001i,Raila has crossed th...","{'labels': ['N/A'], 'note': ''}",NaN,"{'first_done_at': 1529758904000, 'last_updated..."
2,"MTPW000020180409ee4900899,Political risk remai...","{'labels': ['N/A'], 'note': ''}",NaN,"{'first_done_at': 1529688087000, 'last_updated..."
3,"AFNWS00020171114edbe0018o,You Can Still Have Y...","{'labels': ['N/A'], 'note': ''}",NaN,"{'first_done_at': 1529587470000, 'last_updated..."
4,"JMATH00020171219edcj0008f,Fuzzy Logic; Studies...","{'labels': ['N/A'], 'note': ''}",NaN,"{'first_done_at': 1529659936000, 'last_updated..."


In [16]:
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from string import punctuation
from collections import Counter
import numpy as np

#Deleting the metadata, getting only the title
titles = [re.split("^(.+?),", title)[-1] for title in raw_data.content]
#Deleting whitespace
titles = [re.sub(r'\s+', ' ', title) for title in titles]
#Checking for titles without labels
labels = [md['labels'] for md in raw_data.annotation]
labels2 = []
for i in range(0, len(labels)):
    if i:
        labels2.append(labels[i])
    else:
        titles.pop(i)
labels = labels2

#One-hot encoding
distressed_labels = []
not_distressed_labels = []
na_labels = []

for label in labels:
    if "Distress" in label:
        distressed_labels.append(1)
    else:
        distressed_labels.append(0)
        
    if"Not Distress" in label: 
        not_distressed_labels.append(1)
    else:
        not_distressed_labels.append(0)

    if "N/A" in label:
        na_labels.append(1)
    else:
        na_labels.append(0)


encoded_labels = list(zip(distressed_labels, not_distressed_labels, na_labels))
#Dividing the data into testing and training sets
X_train, X_test, y_train, y_test = train_test_split(titles, encoded_labels, test_size=0.20, random_state=42)
dist_train = [item[0] for item in y_train]
not_dist_train = [item[1] for item in y_train]
na_train = [item[2] for item in y_train]

dist_test = [item[0] for item in y_test]
not_dist_test = [item[1] for item in y_test]
na_test = [item[2] for item in y_test]

['Raila has crossed the Rubicon',
 'Political risk remains the key consideration for dealmaking in Africa',
 'You Can Still Have Your Say On Education Bill',
 'Fuzzy Logic; Studies from Faculty of Technology Reveal New Findings on Fuzzy Logic (Active fault tolerant control based on interval type-2 fuzzy sliding mode controller and non linear adaptive observer for 3-DOF laboratory helicopter)',
 'EFF cocks a snoot at council',
 'Woolworths Holdings Limited Notification In Terms Of Section 122(3) Of The Companies Act',
 'US Pulls Out of UN Migrant and Refugee Pact',
 'Theory EXPLODES online that Pharaohs of Ancient Egypt were ALIEN HYBRIDS',
 '"Nigerian youths not lazy, they must vote against Buhari in 2019, Fayose says"',
 'Taxi strike over minister',
 'MICT launches a short film on Gender Based Violence',
 'TETFund launch projects worth N1.4bn at Jigawa varsity',
 'IN PARLIAMENT TODAY',
 'FACTBOX-Banks take hits on exposure to ailing furniture retailer Steinhoff',
 '"Findley, Others Co

In [17]:
#Preparing testing and training dataframes
train_df = pd.DataFrame({
    'text': X_train,
    'distressed': dist_train,
    'not_distressed': not_dist_train,
    'na': na_train,
    'labels':y_train
})

print(train_df.head())

eval_df = pd.DataFrame({
    'text': X_test,
    'distressed': dist_test,
    'not_distressed': not_dist_test,
    'na': na_test,
    'labels': y_test
})

print([eval_df['labels']])

                                                text  distressed  \
0                Government Of Morocco; Water Supply           0   
1                            BoG urges consolidation           0   
2  Groups with ties to Guptas 'breached criminal ...           1   
3  "Kukah, Odinkalu, CSOs take anti-NGO bill prot...           0   
4       Making the land debate about race is a curse           0   

   not_distressed  na     labels  
0               0   1  (0, 0, 1)  
1               1   0  (0, 1, 0)  
2               0   0  (1, 0, 0)  
3               1   0  (0, 1, 0)  
4               1   0  (0, 1, 0)  
[0       (0, 0, 1)
1       (1, 0, 0)
2       (0, 0, 1)
3       (0, 0, 1)
4       (1, 0, 0)
          ...    
2886    (0, 0, 1)
2887    (0, 0, 1)
2888    (0, 1, 0)
2889    (0, 0, 1)
2890    (0, 0, 1)
Name: labels, Length: 2891, dtype: object]


In [18]:
from simpletransformers.classification import MultiLabelClassificationModel

#Preparing the simpletransformers model
model = MultiLabelClassificationModel('roberta', 'roberta-base', num_labels=3, use_cuda=False, args={'output_dir': 'third_outputs/'})

In [19]:
#Training the model
model.train_model(train_df, multi_label=True)

Converting to features started.


Running loss: 0.318165
Training of roberta model complete. Saved to third_outputs/.


In [20]:
#Evaluating the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

Converting to features started.


In [21]:
#Trying out a prediction
model.predict(['High income flux expected.'])

Converting to features started.


([[0, 0, 0]], array([[0.36391598, 0.37992305, 0.18566957]], dtype=float32))

In [22]:
#Viewing evaluation
result

{'LRAP': 0.864147353856797, 'eval_loss': 0.400358904482416}

In [23]:
import numpy as np
from sklearn.metrics import label_ranking_average_precision_score

#A quick implementation of the zero rule algorithm, to compare metrics
def zero_rule_algorithm(train_df, eval_df):
    outputs = [t for t in train_df['labels']]
    most_common = max(set(outputs), key = outputs.count)
    return [most_common for i in range(len(eval_df))]
    
predictions = zero_rule_algorithm(train_df, eval_df)
values = [i for i in eval_df['labels']]
label_ranking_average_precision_score(values, predictions)

0.6890925861870033